In [2]:
import pandas as pd
import numpy as np
import json
import gzip
import os
from os import listdir
import seaborn as sns
import matplotlib.pyplot as plt
import py_entitymatching as em
import py_stringmatching as sm
from scipy.io import arff
from sklearn import preprocessing


In [12]:
#read table to dataframe and create train/test split with equal distribution of labels within table
#example small book
small = pd.read_csv('../../src/data/schemafiltereddata/small_tables.csv')
small_books = small[small['Class']=='Book']
pd.set_option('display.max_columns', None)
small_books

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher,CreativeWork_aggregaterating,CreativeWork_alternativeheadline,CreativeWork_articlebody,CreativeWork_author,CreativeWork_commentcount,CreativeWork_copyrightholder,CreativeWork_copyrightyear,CreativeWork_creator,CreativeWork_datecreated,CreativeWork_datemodified,CreativeWork_datepublished,CreativeWork_genre,CreativeWork_headline,CreativeWork_inlanguage,CreativeWork_interactioncount,CreativeWork_interactionstatistic,CreativeWork_interactiontype,CreativeWork_keywords,CreativeWork_mainentityofpage,CreativeWork_name,CreativeWork_offers,CreativeWork_publisher,CreativeWork_version,Event_doortime,Event_duration,Event_enddate,Event_eventattendancemode,Event_eventstatus,Event_location,Event_name,Event_organizer,Event_performer,Event_performers,Event_typicalagerange,Hotel_address,Hotel_aggregaterating,Hotel_email,Hotel_geo,Hotel_name,Hotel_pricerange,Hotel_starrating,Hotel_telephone,LocalBusiness_address,LocalBusiness_addresslocality,LocalBusiness_addressregion,LocalBusiness_aggregaterating,LocalBusiness_areaserved,LocalBusiness_citystatezip,LocalBusiness_contactpoint,LocalBusiness_email,LocalBusiness_faxnumber,LocalBusiness_founder,LocalBusiness_geo,LocalBusiness_hasmap,LocalBusiness_legalname,LocalBusiness_location,LocalBusiness_name,LocalBusiness_openinghours,LocalBusiness_openinghoursspecification,LocalBusiness_owns,LocalBusiness_paymentaccepted,LocalBusiness_postalcode,LocalBusiness_pricerange,LocalBusiness_review,LocalBusiness_sameas,LocalBusiness_streetaddress,LocalBusiness_telephone,MusicAlbum_byartist,MusicAlbum_name,MusicAlbum_numtracks,MusicAlbum_track,MusicRecording_byartist,MusicRecording_datecreated,MusicRecording_datepublished,MusicRecording_duration,MusicRecording_genre,MusicRecording_inalbum,MusicRecording_name,Person_additionalname,Person_address,Person_affiliation,Person_birthdate,Person_birthplace,Person_contactpoint,Person_deathdate,Person_email,Person_familyname,Person_faxnumber,Person_gender,Person_givenname,Person_height,Person_homelocation,Person_jobtitle,Person_knowslanguage,Person_mainentityofpage,Person_makesoffer,Person_memberof,Person_nationality,Person_sameas,Person_telephone,Person_weight,Person_worklocation,Person_worksfor,Place_geo,Place_name,Place_openinghoursspecification,Place_sameas,Place_telephone,Product_additionalproperty,Product_address,Product_aggregaterating,Product_alternatename,Product_audience,Product_author,Product_availability,Product_bestrating,Product_brand,Product_breadcrumb,Product_category,Product_color,Product_condition,Product_datepublished,Product_depth,Product_disambiguatingdescription,Product_geo,Product_gtin,Product_gtin12,Product_gtin13,Product_gtin14,Product_gtin8,Product_height,Product_identifier,Product_isrelatedto,Product_issimilarto,Product_itemcondition,Product_itemlistelement,Product_mainentityofpage,Product_manufacturer,Product_material,Product_model,Product_mpn,Product_name,Product_offerdetails,Product_offers,Product_price,Product_pricecurrency,Product_ratingvalue,Product_releasedate,Product_review,Product_reviewcount,Product_reviews,Product_shop-currency,Product_title,Product_weight,Product_width,Product_worstrating,Recipe_aggregaterating,Recipe_author,Recipe_cookingmethod,Recipe_cooktime,Recipe_datemodified,Recipe_datepublished,Recipe_headline,Recipe_ingredients,Recipe_ispartof,Recipe_keywords,Recipe_mainentityofpage,Recipe_name,Recipe_nutrition,Recipe_performtime,Recipe_preptime,Recipe_publisher,Recipe_recipecategory,Recipe_recipecuisine,Recipe_recipeingredient,Recipe_recipeinstructions,Recipe_recipeyield,Recipe_review,Recipe_suitablefordiet,Recipe_totaltime,Restaurant_acceptsreservations,Restaurant_address,Restaurant_aggregaterating,Restaurant_geo,Restaurant_hasmenu,Restaura

In [10]:
small_books = small_books.drop(small_books.columns[21:], axis=1)
small_books

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,...,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher
0,Book_aardvark-books.com_September2020.json.gz,Book,26.0,9.0,7.0,5,0.027473,0.027473,35.0,0,...,0,1,1,0,0,1,1,0,1,1
1,Book_americanreadingathome.com_September2020.json.gz,Book,33.0,9.0,5.0,3,0.018182,0.018182,15.0,0,...,0,0,0,0,1,0,1,1,1,0
2,Book_barmatrioshka.com_September2020.json.gz,Book,35.0,11.0,9.0,0,0.000000,0.000000,0.0,1,...,0,1,1,0,1,0,1,1,1,1
3,Book_bibliobeat.com_September2020.json.gz,Book,22.0,10.0,6.0,0,0.000000,0.000000,0.0,0,...,0,1,1,0,1,1,1,0,0,0
4,Book_bookapy.com_September2020.json.gz,Book,26.0,13.0,8.0,11,0.052885,0.052885,88.0,0,...,0,1,1,0,1,1,1,1,1,0
5,Book_books.org_September2020.json.gz,Book,36.0,8.0,5.0,11,0.061111,0.061111,55.0,0,...,0,0,0,0,0,1,1,1,0,1
6,Book_capitalsoftworks.co.uk_September2020.json.gz,Book,25.0,13.0,7.0,17,0.097143,0.097143,119.0,0,...,0,1,0,0,1,1,1,1,0,1
7,Book_carlosruizzafon.co.uk_September2020.json.gz,Book,27.0,10.0,5.0,1,0.007407,0.007407,5.0,0,...,0,0,1,1,0,1,1,0,0,0
8,Book_clairenorth.com_September2020.json.gz,Book,22.0,11.0,6.0,0,0.000000,0.000000,0.0,0,...,0,0,1,1,0,1,1,0,0,1
9,Book_cressidacowell.co.uk_September2020.json.gz,Book,21.0,11.0,6.0,0,0.000000,0.000000,0.0,0,...,0,0,1,1,0,1,1,0,0,1


In [14]:
book_col = [col for col in small_books if col.startswith('Book_')]
book_col

['Book_aggregaterating',
 'Book_author',
 'Book_bookedition',
 'Book_bookformat',
 'Book_datepublished',
 'Book_genre',
 'Book_inlanguage',
 'Book_isbn',
 'Book_name',
 'Book_numberofpages',
 'Book_offers',
 'Book_publisher']

In [ ]:
X = small_books
train_size = int(len(X) * 0.7)
train, test = X[0:train_size], X[(train_size:len(X)) & ]
#X_train, y_train, X_test, y_test = iterative_train_test_split(train,test, test_size = 0.2)
#test = test.values
print('Total Counts: %d' % (len(X)))
print('Train Counts: %d' % (len(train)))
print('Testing Counts: %d' % (len(test)))

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split
X = small_books
y 
t_train, y_train, t_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)

In [3]:
path='/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/schemafiltereddata/Book/'

In [ ]:
test_books = pd.read_json(path+'Book_aardvark-books.com_September2020.json.gz', compression='gzip', lines=True)
test_books

In [33]:
em.is_dfinfo_present(test)

True

In [31]:
test_books.to_csv('test_csv.csv')
test = em.read_csv_metadata('test_csv.csv')
test['ID'] = range(0, len(test))
em.set_key(test, 'ID')

Metadata file is not present in the given path; proceeding to read the csv file.


True

In [ ]:
train_test = em.split_train_test(test, train_proportion=0.5)
train, test = train_test['train'], train_test['test']

In [ ]:
from skmultilearn.dataset import load_dataset
#X,y, _, _ = load_dataset(small_books)

In [68]:
X = test_books
train_size = int(len(X) * 0.7)
train, test = X[0:train_size], X[train_size:len(X)]
#X_train, y_train, X_test, y_test = iterative_train_test_split(train,test, test_size = 0.2)
#test = test.values
print('Total Counts: %d' % (len(X)))
print('Train Counts: %d' % (len(train)))
print('Testing Counts: %d' % (len(test)))

Total Counts: 26
Train Counts: 18
Testing Counts: 8


In [69]:
rel = pd.read_csv('../../src/data/data/CSV_files/relevant_tables.csv')
rel

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher,CreativeWork_aggregaterating,CreativeWork_alternativeheadline,CreativeWork_articlebody,CreativeWork_author,CreativeWork_commentcount,CreativeWork_copyrightholder,CreativeWork_copyrightyear,CreativeWork_creator,CreativeWork_datecreated,CreativeWork_datemodified,CreativeWork_datepublished,CreativeWork_genre,CreativeWork_headline,CreativeWork_inlanguage,CreativeWork_interactioncount,CreativeWork_interactionstatistic,CreativeWork_interactiontype,CreativeWork_keywords,CreativeWork_mainentityofpage,CreativeWork_name,CreativeWork_offers,CreativeWork_publisher,CreativeWork_version,Event_doortime,Event_duration,Event_enddate,Event_eventattendancemode,Event_eventstatus,Event_location,Event_name,Event_organizer,Event_performer,Event_performers,Event_typicalagerange,Hotel_address,Hotel_aggregaterating,Hotel_email,Hotel_geo,Hotel_name,Hotel_pricerange,Hotel_starrating,Hotel_telephone,LocalBusiness_address,LocalBusiness_addresslocality,LocalBusiness_addressregion,LocalBusiness_aggregaterating,LocalBusiness_areaserved,LocalBusiness_citystatezip,LocalBusiness_contactpoint,LocalBusiness_email,LocalBusiness_faxnumber,LocalBusiness_founder,LocalBusiness_geo,LocalBusiness_hasmap,LocalBusiness_legalname,LocalBusiness_location,LocalBusiness_name,LocalBusiness_openinghours,LocalBusiness_openinghoursspecification,LocalBusiness_owns,LocalBusiness_paymentaccepted,LocalBusiness_postalcode,LocalBusiness_pricerange,LocalBusiness_review,LocalBusiness_sameas,LocalBusiness_streetaddress,LocalBusiness_telephone,MusicAlbum_byartist,MusicAlbum_name,MusicAlbum_numtracks,MusicAlbum_track,MusicRecording_byartist,MusicRecording_datecreated,MusicRecording_datepublished,MusicRecording_duration,MusicRecording_genre,MusicRecording_inalbum,MusicRecording_name,Person_additionalname,Person_address,Person_affiliation,Person_birthdate,Person_birthplace,Person_contactpoint,Person_deathdate,Person_email,Person_familyname,Person_faxnumber,Person_gender,Person_givenname,Person_height,Person_homelocation,Person_jobtitle,Person_knowslanguage,Person_mainentityofpage,Person_makesoffer,Person_memberof,Person_nationality,Person_sameas,Person_telephone,Person_weight,Person_worklocation,Person_worksfor,Place_geo,Place_name,Place_openinghoursspecification,Place_sameas,Place_telephone,Product_additionalproperty,Product_address,Product_aggregaterating,Product_alternatename,Product_audience,Product_author,Product_availability,Product_bestrating,Product_brand,Product_breadcrumb,Product_category,Product_color,Product_condition,Product_datepublished,Product_depth,Product_disambiguatingdescription,Product_geo,Product_gtin,Product_gtin12,Product_gtin13,Product_gtin14,Product_gtin8,Product_height,Product_identifier,Product_isrelatedto,Product_issimilarto,Product_itemcondition,Product_itemlistelement,Product_mainentityofpage,Product_manufacturer,Product_material,Product_model,Product_mpn,Product_name,Product_offerdetails,Product_offers,Product_price,Product_pricecurrency,Product_ratingvalue,Product_releasedate,Product_review,Product_reviewcount,Product_reviews,Product_shop-currency,Product_title,Product_weight,Product_width,Product_worstrating,Recipe_aggregaterating,Recipe_author,Recipe_cookingmethod,Recipe_cooktime,Recipe_datemodified,Recipe_datepublished,Recipe_headline,Recipe_ingredients,Recipe_ispartof,Recipe_keywords,Recipe_mainentityofpage,Recipe_name,Recipe_nutrition,Recipe_performtime,Recipe_preptime,Recipe_publisher,Recipe_recipecategory,Recipe_recipecuisine,Recipe_recipeingredient,Recipe_recipeinstructions,Recipe_recipeyield,Recipe_review,Recipe_suitablefordiet,Recipe_totaltime,Restaurant_acceptsreservations,Restaurant_address,Restaurant_aggregaterating,Restaurant_geo,Restaurant_hasmenu,Restaura

In [74]:
columns = list(rel.columns[9:])
for i in columns:
    if rel[i].sum()<50:
        print(i)
        print (rel[i].sum())
    # printing the third element of the column
    #print (rel[i].sum()<50)

CreativeWork_creator
45
Event_duration
32
LocalBusiness_founder
17
LocalBusiness_owns
2
Person_mainentityofpage
16
Product_itemlistelement
18
Product_offerdetails
18
Product_reviews
20
Restaurant_menu
26


In [63]:
full_table = pd.read_csv('../../src/data/data/CSV_files/dummy_overview_all.csv')
full_table

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher,CreativeWork_aggregaterating,CreativeWork_alternativeheadline,CreativeWork_articlebody,CreativeWork_author,CreativeWork_commentcount,CreativeWork_copyrightholder,CreativeWork_copyrightyear,CreativeWork_creator,CreativeWork_datecreated,CreativeWork_datemodified,CreativeWork_datepublished,CreativeWork_genre,CreativeWork_headline,CreativeWork_inlanguage,CreativeWork_interactioncount,CreativeWork_interactionstatistic,CreativeWork_interactiontype,CreativeWork_keywords,CreativeWork_mainentityofpage,CreativeWork_name,CreativeWork_offers,CreativeWork_publisher,CreativeWork_version,Event_doortime,Event_duration,Event_enddate,Event_eventattendancemode,Event_eventstatus,Event_location,Event_name,Event_organizer,Event_performer,Event_performers,Event_typicalagerange,Hotel_address,Hotel_aggregaterating,Hotel_email,Hotel_geo,Hotel_name,Hotel_pricerange,Hotel_starrating,Hotel_telephone,LocalBusiness_address,LocalBusiness_addresslocality,LocalBusiness_addressregion,LocalBusiness_aggregaterating,LocalBusiness_areaserved,LocalBusiness_citystatezip,LocalBusiness_contactpoint,LocalBusiness_email,LocalBusiness_faxnumber,LocalBusiness_founder,LocalBusiness_geo,LocalBusiness_hasmap,LocalBusiness_legalname,LocalBusiness_location,LocalBusiness_name,LocalBusiness_openinghours,LocalBusiness_openinghoursspecification,LocalBusiness_owns,LocalBusiness_paymentaccepted,LocalBusiness_postalcode,LocalBusiness_pricerange,LocalBusiness_review,LocalBusiness_sameas,LocalBusiness_streetaddress,LocalBusiness_telephone,MusicAlbum_byartist,MusicAlbum_name,MusicAlbum_numtracks,MusicAlbum_track,MusicRecording_byartist,MusicRecording_datecreated,MusicRecording_datepublished,MusicRecording_duration,MusicRecording_genre,MusicRecording_inalbum,MusicRecording_name,Person_additionalname,Person_address,Person_affiliation,Person_birthdate,Person_birthplace,Person_contactpoint,Person_deathdate,Person_email,Person_familyname,Person_faxnumber,Person_gender,Person_givenname,Person_height,Person_homelocation,Person_jobtitle,Person_knowslanguage,Person_mainentityofpage,Person_makesoffer,Person_memberof,Person_nationality,Person_sameas,Person_telephone,Person_weight,Person_worklocation,Person_worksfor,Place_geo,Place_name,Place_openinghoursspecification,Place_sameas,Place_telephone,Product_additionalproperty,Product_address,Product_aggregaterating,Product_alternatename,Product_audience,Product_author,Product_availability,Product_bestrating,Product_brand,Product_breadcrumb,Product_category,Product_color,Product_condition,Product_datepublished,Product_depth,Product_disambiguatingdescription,Product_geo,Product_gtin,Product_gtin12,Product_gtin13,Product_gtin14,Product_gtin8,Product_height,Product_identifier,Product_isrelatedto,Product_issimilarto,Product_itemcondition,Product_itemlistelement,Product_mainentityofpage,Product_manufacturer,Product_material,Product_model,Product_mpn,Product_name,Product_offerdetails,Product_offers,Product_price,Product_pricecurrency,Product_ratingvalue,Product_releasedate,Product_review,Product_reviewcount,Product_reviews,Product_shop-currency,Product_title,Product_weight,Product_width,Product_worstrating,Recipe_aggregaterating,Recipe_author,Recipe_cookingmethod,Recipe_cooktime,Recipe_datemodified,Recipe_datepublished,Recipe_headline,Recipe_ingredients,Recipe_ispartof,Recipe_keywords,Recipe_mainentityofpage,Recipe_name,Recipe_nutrition,Recipe_performtime,Recipe_preptime,Recipe_publisher,Recipe_recipecategory,Recipe_recipecuisine,Recipe_recipeingredient,Recipe_recipeinstructions,Recipe_recipeyield,Recipe_review,Recipe_suitablefordiet,Recipe_totaltime,Restaurant_acceptsreservations,Restaurant_address,Restaurant_aggregaterating,Restaurant_geo,Restaurant_hasmenu,Restaura

In [67]:
relevant =full_table[(full_table['num_rows']>10) & (full_table['percentage_table_NA']<0.20)]
relevant

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher,CreativeWork_aggregaterating,CreativeWork_alternativeheadline,CreativeWork_articlebody,CreativeWork_author,CreativeWork_commentcount,CreativeWork_copyrightholder,CreativeWork_copyrightyear,CreativeWork_creator,CreativeWork_datecreated,CreativeWork_datemodified,CreativeWork_datepublished,CreativeWork_genre,CreativeWork_headline,CreativeWork_inlanguage,CreativeWork_interactioncount,CreativeWork_interactionstatistic,CreativeWork_interactiontype,CreativeWork_keywords,CreativeWork_mainentityofpage,CreativeWork_name,CreativeWork_offers,CreativeWork_publisher,CreativeWork_version,Event_doortime,Event_duration,Event_enddate,Event_eventattendancemode,Event_eventstatus,Event_location,Event_name,Event_organizer,Event_performer,Event_performers,Event_typicalagerange,Hotel_address,Hotel_aggregaterating,Hotel_email,Hotel_geo,Hotel_name,Hotel_pricerange,Hotel_starrating,Hotel_telephone,LocalBusiness_address,LocalBusiness_addresslocality,LocalBusiness_addressregion,LocalBusiness_aggregaterating,LocalBusiness_areaserved,LocalBusiness_citystatezip,LocalBusiness_contactpoint,LocalBusiness_email,LocalBusiness_faxnumber,LocalBusiness_founder,LocalBusiness_geo,LocalBusiness_hasmap,LocalBusiness_legalname,LocalBusiness_location,LocalBusiness_name,LocalBusiness_openinghours,LocalBusiness_openinghoursspecification,LocalBusiness_owns,LocalBusiness_paymentaccepted,LocalBusiness_postalcode,LocalBusiness_pricerange,LocalBusiness_review,LocalBusiness_sameas,LocalBusiness_streetaddress,LocalBusiness_telephone,MusicAlbum_byartist,MusicAlbum_name,MusicAlbum_numtracks,MusicAlbum_track,MusicRecording_byartist,MusicRecording_datecreated,MusicRecording_datepublished,MusicRecording_duration,MusicRecording_genre,MusicRecording_inalbum,MusicRecording_name,Person_additionalname,Person_address,Person_affiliation,Person_birthdate,Person_birthplace,Person_contactpoint,Person_deathdate,Person_email,Person_familyname,Person_faxnumber,Person_gender,Person_givenname,Person_height,Person_homelocation,Person_jobtitle,Person_knowslanguage,Person_mainentityofpage,Person_makesoffer,Person_memberof,Person_nationality,Person_sameas,Person_telephone,Person_weight,Person_worklocation,Person_worksfor,Place_geo,Place_name,Place_openinghoursspecification,Place_sameas,Place_telephone,Product_additionalproperty,Product_address,Product_aggregaterating,Product_alternatename,Product_audience,Product_author,Product_availability,Product_bestrating,Product_brand,Product_breadcrumb,Product_category,Product_color,Product_condition,Product_datepublished,Product_depth,Product_disambiguatingdescription,Product_geo,Product_gtin,Product_gtin12,Product_gtin13,Product_gtin14,Product_gtin8,Product_height,Product_identifier,Product_isrelatedto,Product_issimilarto,Product_itemcondition,Product_itemlistelement,Product_mainentityofpage,Product_manufacturer,Product_material,Product_model,Product_mpn,Product_name,Product_offerdetails,Product_offers,Product_price,Product_pricecurrency,Product_ratingvalue,Product_releasedate,Product_review,Product_reviewcount,Product_reviews,Product_shop-currency,Product_title,Product_weight,Product_width,Product_worstrating,Recipe_aggregaterating,Recipe_author,Recipe_cookingmethod,Recipe_cooktime,Recipe_datemodified,Recipe_datepublished,Recipe_headline,Recipe_ingredients,Recipe_ispartof,Recipe_keywords,Recipe_mainentityofpage,Recipe_name,Recipe_nutrition,Recipe_performtime,Recipe_preptime,Recipe_publisher,Recipe_recipecategory,Recipe_recipecuisine,Recipe_recipeingredient,Recipe_recipeinstructions,Recipe_recipeyield,Recipe_review,Recipe_suitablefordiet,Recipe_totaltime,Restaurant_acceptsreservations,Restaurant_address,Restaurant_aggregaterating,Restaurant_geo,Restaurant_hasmenu,Restaura

In [66]:
columns = list(full_table.columns[9:])
for i in columns:
    if full_table[i].sum()<50:
        print (rel[i].sum())
        print(i)
    # printing the third element of the column
    #

2
LocalBusiness_owns
18
Product_offerdetails
